In [5]:
from pptx import Presentation
import pandas as pd
import matplotlib.pyplot as plt
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
import os, io
from datetime import datetime

## Input

In [141]:
ppt_save_path = "E:/pyDev/output/"
model_name = "YOUR MODEL"
features = ["ORDERS","LINES","PCS","SKU","CITIES"]

model_intro = """The term machine learning was first coined in the 1950s when Artificial Intelligence pioneer Arthur Samuel built the first self-learning system for playing checkers. He noticed that the more the system played, the better it performed.

Fueled by advances in statistics and computer science, as well as better datasets and the growth of neural networks, machine learning has truly taken off in recent years.

Today, whether you realize it or not, machine learning is everywhere ‒ automated translation, image recognition, voice search technology, self-driving cars, and beyond."""

## Data

In [180]:
tdf = pd.read_csv("C:\\Users\\Admin\\Downloads\\automate-powerpoint-main\\automate-powerpoint-main\\volumes per day.csv")
tdf.head(2)

,Unnamed: 0,DATE,WEEK,DAY,ORDERS,LINES,PCS,SKU,CITIES
0,0,2017-01-02,WEEK-1,MON,776,1367,1595.0,487,174
1,1,2017-01-03,WEEK-1,TUE,902,1550,1861.0,547,188


In [186]:
summDF = tdf.describe()
feature_imp = summDF.copy()
feature_imp['features'] = summDF.index
feature_imp

,Unnamed: 0,ORDERS,LINES,PCS,SKU,CITIES,features
count,30.000000,30.000000,30.000000,30.000000,30.00000,30.000000,count
mean,14.500000,1062.466667,2040.966667,2385.600000,534.70000,211.566667,mean
std,8.803408,414.971103,794.702394,951.366516,111.64017,39.510525,std
min,0.000000,278.000000,550.000000,610.000000,243.00000,123.000000,min
25%,7.250000,790.500000,1557.000000,1825.000000,472.50000,189.000000,25%
50%,14.500000,1022.000000,1966.000000,2334.500000,540.00000,207.500000,50%
75%,21.750000,1431.750000,2698.500000,3126.750000,618.75000,248.250000,75%
max,29.000000,1921.000000,3479.000000,4188.000000,694.00000,279.000000,max


## Generate presentation

In [208]:
class generatePPT:
    
    def __init__(self, ModelName, out_path, feature_imp, summDF, features, model_intro):
        
        self.model_name = ModelName
        self.t_date = str(datetime.now().date())
        self.pptfile = Presentation()
        self.out_path = out_path
        self.model_intro = model_intro
        self.summDF = summDF
        self.features = features
        self.feature_imp = feature_imp
        
    def addTitlePage(self):

        title_slide_layout = self.pptfile.slide_layouts[0]
        slide = self.pptfile.slides.add_slide(title_slide_layout)
        title = slide.shapes.title
        subtitle = slide.placeholders[1]

        title.text = (self.model_name + " Refresh").title()
        subtitle.text = self.t_date
        
    def addModelIntroduction(self):
        
        title_only_slide_layout = self.pptfile.slide_layouts[5]
        slide = self.pptfile.slides.add_slide(title_only_slide_layout)
        slide.shapes.title.text = self.model_name
        
        left, top, width, height = Inches(1),Inches(1),Inches(8), Inches(8)
        txBox = slide.shapes.add_textbox(left, top, width, height)
        tf = txBox.text_frame
        tf.word_wrap = True
        

        p = tf.add_paragraph()
        p.text = "\n"+self.model_intro
        p.font.size = Pt(12)
        
    def addDataSummary(self):
        
        def fillTable(df, tbl):
            colNames = list(df.columns)

            for Ci in range(df.shape[1]):
                tbl.cell(0,Ci).text = str(colNames[Ci])
                tbl.cell(0,Ci).text_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT

            for Ri in range(df.shape[0]):
                for Ci in range(df.shape[1]):
                    tbl.cell(Ri+1,Ci).text = str(round(df.iloc[Ri,Ci],2))
                    tbl.cell(Ri+1,Ci).text_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT

            return tbl
    
        title_only_slide_layout = self.pptfile.slide_layouts[5]
        slide = self.pptfile.slides.add_slide(title_only_slide_layout)
        
        
        left, top, width, height = Inches(1), Inches(2), Inches(8), Inches(1.5)
        tabRc, tabCc = self.summDF.shape[0]+1, summDF.shape[1]

        slide.shapes.title.text = "Data Summary"
        shape = slide.shapes.add_table(tabRc, tabCc, left, top, width, height)
        table = shape.table
        table = fillTable(self.summDF, table)
    
    def addPDPplots(self):
        
        def genAndSavePlot(col):
            plt.plot(summDF[col])
            plt.savefig('tmp_plt.png', bbox_inches='tight')
            plt.clf()
            
        for col in self.features:
            genAndSavePlot(col)
            img = "tmp_plt.png"
            
            title_only_slide_layout = self.pptfile.slide_layouts[5]
            slide = self.pptfile.slides.add_slide(title_only_slide_layout)
            slide.shapes.title.text = "PDP Plot - "+col

            left, top, width, height = Inches(1), Inches(1.5), Inches(6), Inches(5)
            pic = slide.shapes.add_picture(img, left, top, width, height)

    def getFeatureImportance(self):
        
        title_only_slide_layout = self.pptfile.slide_layouts[5]
        slide = self.pptfile.slides.add_slide(title_only_slide_layout)
        slide.shapes.title.text = "Feature Importance"
        
        ########## UPDATE THIS WITH CODE ###############################
        plt.barh(self.feature_imp["features"],self.feature_imp["ORDERS"])
        plt.savefig('tmp_plt.png', bbox_inches='tight')
        plt.clf()

        img = "tmp_plt.png"
        left, top, width, height = Inches(1), Inches(1.5), Inches(7), Inches(5)
        pic = slide.shapes.add_picture(img, left, top, width, height)
    
    def savePpt(self):
        self.pptfile.save(self.out_path+"_".join(self.model_name.split())+"_refresh.pptx")

In [209]:
pptObj = generatePPT(model_name, ppt_save_path, feature_imp, summDF, features, model_intro)

In [210]:
pptObj.addTitlePage()
pptObj.addModelIntroduction()
pptObj.addDataSummary()
pptObj.addPDPplots()
pptObj.getFeatureImportance()

<Figure size 640x480 with 0 Axes>

In [211]:
pptObj.savePpt()